---
sidebar_label: Reka
---

# ChatReka

本笔记本提供了使用 Reka [聊天模型](../../concepts/chat_models.mdx) 的快速入门指南。

Reka 有多种聊天模型。您可以在 [Reka 文档](https://docs.reka.ai/available-models) 中找到有关其最新模型及其成本、上下文窗口和支持的输入类型的信息。




## 概览
### 集成详情

| 类 | 包 | 本地 | 可序列化 | JS 支持 | 包下载 | 包最新版本 |
| :--- | :--- | :---: | :---: |  :---: | :---: | :---: |
| [ChatReka] | [langchain_community](https://python.langchain.com/api_reference/community/index.html) | ✅ | ❌ | ❌ | ![PyPI - Downloads](https://img.shields.io/pypi/dm/langchain_community?style=flat-square&label=%20) | ![PyPI - Version](https://img.shields.io/pypi/v/langchain_community?style=flat-square&label=%20) |

### 模型功能
| [Tool calling](/docs/how_to/tool_calling) | [Structured output](/docs/how_to/structured_output/) | JSON 模式 | [Image input](/docs/how_to/multimodal_inputs/) | Audio input | Video input | [Token-level streaming](/docs/how_to/chat_streaming/) | Native async | [Token usage](/docs/how_to/chat_token_usage_tracking/) | [Logprobs](/docs/how_to/logprobs/) |
| :---: | :---: | :---: | :---: |  :---: | :---: | :---: | :---: | :---: | :---: |
| ✅ | ❌ | ❌ | ✅ | ✅ | ✅ | ✅ | ✅ | ❌ | ❌ |

## 设置

要访问 Reka 模型，您需要创建一个 Reka 开发者账户，获取 API 密钥，然后安装 `langchain_community` 集成包以及 reka Python 包，方法是使用 'pip install reka-api' 进行安装。

### 凭证

前往 https://platform.reka.ai/ 注册 Reka 并生成 API 密钥。完成此操作后，设置 REKA_API_KEY 环境变量：

### 安装

LangChain 的 __ModuleName__ 集成位于 `langchain_community` 包中：

In [4]:
%pip install -qU langchain_community reka-api

Note: you may need to restart the kernel to use updated packages.


## 实例化

In [1]:
import getpass
import os

os.environ["REKA_API_KEY"] = getpass.getpass("Enter your Reka API key: ")

可选：使用 Langsmith 跟踪模型的执行

In [ ]:
import getpass
import os

os.environ["LANGSMITH_TRACING"] = "true"
os.environ["LANGSMITH_API_KEY"] = getpass.getpass("Enter your Langsmith API key: ")

In [1]:
from langchain_community.chat_models import ChatReka

model = ChatReka()

## 调用

In [2]:
model.invoke("hi")

AIMessage(content=' Hello! How can I help you today? If you have a question, need assistance, or just want to chat, feel free to let me know. Have a great day!\n\n', additional_kwargs={}, response_metadata={}, id='run-61522ec2-0587-4fd5-a492-5b205fd8860c-0')

# 图片输入

In [3]:
from langchain_core.messages import HumanMessage

image_url = "https://v0.docs.reka.ai/_images/000000245576.jpg"

message = HumanMessage(
    content=[
        {"type": "text", "text": "describe the weather in this image"},
        {
            "type": "image_url",
            "image_url": {"url": image_url},
        },
    ],
)
response = model.invoke([message])
print(response.content)

 The image shows an indoor setting with no visible windows or natural light, and there are no indicators of weather conditions. The focus is on a cat sitting on a computer keyboard, and the background includes a computer monitor and various office supplies.


# 多个图像作为输入

In [4]:
message = HumanMessage(
    content=[
        {"type": "text", "text": "What are the difference between the two images? "},
        {
            "type": "image_url",
            "image_url": {
                "url": "https://cdn.pixabay.com/photo/2019/07/23/13/51/shepherd-dog-4357790_1280.jpg"
            },
        },
        {
            "type": "image_url",
            "image_url": {
                "url": "https://cdn.pixabay.com/photo/2024/02/17/00/18/cat-8578562_1280.jpg"
            },
        },
    ],
)
response = model.invoke([message])
print(response.content)

 The first image features two German Shepherds, one adult and one puppy, in a vibrant, lush green setting. The adult dog is carrying a large stick in its mouth, running through what appears to be a grassy field, with the puppy following close behind. Both dogs exhibit striking physical characteristics typical of the breed, such as pointed ears and dense fur.

The second image shows a close-up of a single cat with striking blue eyes, likely a breed like the Siberian or Maine Coon, in a natural outdoor setting. The cat's fur is lighter, possibly a mix of white and gray, and it has a more subdued expression compared to the dogs. The background is blurred, suggesting a focus on the cat's face.

Overall, the differences lie in the subjects (two dogs vs. one cat), the setting (lush, vibrant grassy field vs. a more muted outdoor background), and the overall mood and activity depicted (playful and active vs. serene and focused).


## 链式调用

In [5]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate(
    [
        (
            "system",
            "You are a helpful assistant that translates {input_language} to {output_language}.",
        ),
        ("human", "{input}"),
    ]
)

chain = prompt | model
chain.invoke(
    {
        "input_language": "English",
        "output_language": "German",
        "input": "I love programming.",
    }
)

AIMessage(content=' Ich liebe Programmieren.\n\n', additional_kwargs={}, response_metadata={}, id='run-ffc4ace1-b73a-4fb3-ad0f-57e60a0f9b8d-0')

通过 Tavily api 搜索使用

# 工具使用与代理创建

## 定义工具

首先，我们需要创建想要使用的工具。我们主要选择的工具将是 Tavily——一个搜索引擎。LangChain 中内置了一个工具，可以轻松地将 Tavily 搜索引擎用作工具。

In [ ]:
import getpass
import os

os.environ["TAVILY_API_KEY"] = getpass.getpass("Enter your Tavily API key: ")

In [6]:
from langchain_community.tools.tavily_search import TavilySearchResults

search = TavilySearchResults(max_results=2)
search_results = search.invoke("what is the weather in SF")
print(search_results)
# If we want, we can create other tools.
# Once we have all the tools we want, we can put them in a list that we will reference later.
tools = [search]

[{'url': 'https://www.weatherapi.com/', 'content': "{'location': {'name': 'San Francisco', 'region': 'California', 'country': 'United States of America', 'lat': 37.775, 'lon': -122.4183, 'tz_id': 'America/Los_Angeles', 'localtime_epoch': 1730484342, 'localtime': '2024-11-01 11:05'}, 'current': {'last_updated_epoch': 1730484000, 'last_updated': '2024-11-01 11:00', 'temp_c': 11.1, 'temp_f': 52.0, 'is_day': 1, 'condition': {'text': 'Mist', 'icon': '//cdn.weatherapi.com/weather/64x64/day/143.png', 'code': 1030}, 'wind_mph': 2.9, 'wind_kph': 4.7, 'wind_degree': 247, 'wind_dir': 'WSW', 'pressure_mb': 1019.0, 'pressure_in': 30.08, 'precip_mm': 0.0, 'precip_in': 0.0, 'humidity': 100, 'cloud': 100, 'feelslike_c': 11.1, 'feelslike_f': 52.0, 'windchill_c': 10.3, 'windchill_f': 50.5, 'heatindex_c': 10.8, 'heatindex_f': 51.5, 'dewpoint_c': 10.4, 'dewpoint_f': 50.6, 'vis_km': 2.8, 'vis_miles': 1.0, 'uv': 3.0, 'gust_mph': 3.8, 'gust_kph': 6.1}}"}, {'url': 'https://weatherspark.com/h/m/557/2024/1/Hist

现在我们可以看到启用此模型进行工具调用的效果了。为了实现这一点，我们使用 `.bind_tools` 来让语言模型了解这些工具。

In [7]:
model_with_tools = model.bind_tools(tools)

现在我们可以调用模型了。我们首先用一个普通的消息来调用它，看看它如何响应。我们可以查看 content 字段和 tool_calls 字段。

In [8]:
from langchain_core.messages import HumanMessage

response = model_with_tools.invoke([HumanMessage(content="Hi!")])

print(f"ContentString: {response.content}")
print(f"ToolCalls: {response.tool_calls}")

ContentString:  Hello! How can I help you today? If you have a question or need information on a specific topic, feel free to ask. Just type your search query and I'll do my best to assist using the available function.


ToolCalls: []


现在，让我们用一些需要调用工具的输入来测试一下。

In [9]:
response = model_with_tools.invoke([HumanMessage(content="What's the weather in SF?")])

print(f"ContentString: {response.content}")
print(f"ToolCalls: {response.tool_calls}")

ContentString: 
ToolCalls: [{'name': 'tavily_search_results_json', 'args': {'query': 'weather in SF'}, 'id': '2548c622-3553-42df-8220-39fde0632bdb', 'type': 'tool_call'}]


我们可以看到现在没有文本内容了，但是有一个工具调用！它要求我们调用 Tavily Search 工具。

这还不是实际调用该工具——它只是告诉我们要调用。为了实际调用它，我们需要创建我们的 agent。

# 创建 Agent

现在已经定义了工具和 LLM，我们可以创建代理。我们将使用 LangGraph 来构建代理。目前，我们正在使用一个高层接口来构建代理，但 LangGraph 的好处在于，这个高层接口背后有一个底层、高度可控的 API，以防你想修改代理逻辑。

现在，我们可以使用 LLM 和工具来初始化代理。

请注意，我们传入的是 model，而不是 model_with_tools。这是因为 `create_react_agent` 会在底层为我们调用 `.bind_tools`。

In [14]:
from langgraph.prebuilt import create_react_agent

agent_executor = create_react_agent(model, tools)

让我们现在在一个应该调用该工具的示例上进行尝试

In [15]:
response = agent_executor.invoke({"messages": [HumanMessage(content="hi!")]})

response["messages"]

[HumanMessage(content='hi!', additional_kwargs={}, response_metadata={}, id='0ab1f3c7-9079-42d4-8a8a-13af5f6c226b'),
 AIMessage(content=' Hello! How can I help you today? If you have a question or need information on a specific topic, feel free to ask. For example, you can start with a search query like "latest news on climate change" or "biography of Albert Einstein".\n\n', additional_kwargs={}, response_metadata={}, id='run-276d9dcd-13f3-481d-b562-8fe3962d9ba1-0')]

为了确切地了解幕后发生了什么（并确保它没有调用工具），我们可以查看 LangSmith 跟踪记录：https://smith.langchain.com/public/2372d9c5-855a-45ee-80f2-94b63493563d/r

In [16]:
response = agent_executor.invoke(
    {"messages": [HumanMessage(content="whats the weather in sf?")]}
)
response["messages"]

[HumanMessage(content='whats the weather in sf?', additional_kwargs={}, response_metadata={}, id='af276c61-3df7-4241-8cb0-81d1f1477bb3'),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': '86da84b8-0d44-444f-8448-7f134f9afa41', 'type': 'function', 'function': {'name': 'tavily_search_results_json', 'arguments': '{"query": "weather in SF"}'}}]}, response_metadata={}, id='run-abe1b8e2-98a6-4f69-8f95-278ac8c141ff-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'weather in SF'}, 'id': '86da84b8-0d44-444f-8448-7f134f9afa41', 'type': 'tool_call'}]),
 ToolMessage(content='[{"url": "https://www.weatherapi.com/", "content": "{\'location\': {\'name\': \'San Francisco\', \'region\': \'California\', \'country\': \'United States of America\', \'lat\': 37.775, \'lon\': -122.4183, \'tz_id\': \'America/Los_Angeles\', \'localtime_epoch\': 1730483436, \'localtime\': \'2024-11-01 10:50\'}, \'current\': {\'last_updated_epoch\': 1730483100, \'last_updated\': \'2024-11-

我们可以查看 LangSmith trace，以确保它有效地调用了搜索工具。

https://smith.langchain.com/public/013ef704-654b-4447-8428-637b343d646e/r

我们已经了解了如何使用 `.invoke` 调用代理以获得最终响应。如果代理执行多个步骤，这可能需要一些时间。为了展示中间进度，我们可以流式传输消息以指示进展。

In [18]:
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="whats the weather in sf?")]}
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': '2457d3ea-f001-4b8c-a1ed-3dc3d1381639', 'type': 'function', 'function': {'name': 'tavily_search_results_json', 'arguments': '{"query": "weather in San Francisco"}'}}]}, response_metadata={}, id='run-0363deab-84d2-4319-bb1e-b55b47fe2274-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'weather in San Francisco'}, 'id': '2457d3ea-f001-4b8c-a1ed-3dc3d1381639', 'type': 'tool_call'}])]}}
----
{'tools': {'messages': [ToolMessage(content='[{"url": "https://www.weatherapi.com/", "content": "{\'location\': {\'name\': \'San Francisco\', \'region\': \'California\', \'country\': \'United States of America\', \'lat\': 37.775, \'lon\': -122.4183, \'tz_id\': \'America/Los_Angeles\', \'localtime_epoch\': 1730483636, \'localtime\': \'2024-11-01 10:53\'}, \'current\': {\'last_updated_epoch\': 1730483100, \'last_updated\': \'2024-11-01 10:45\', \'temp_c\': 11.4, \'temp_f\': 52.5, \'is_day\': 1, 

## API 参考

# 快速入门

Reka AI 的 Reka Fire 是一种可用于推理的